① Tool Definitions + Messages
개발자가 사용할 수 있는 함수(예: get_weather(location))를 미리 정의
사용자가 질문: “What’s the weather in Paris?”

② Tool Calls  
모델이 질문을 보고 “아, 이건 get_weather("paris") 함수를 호출해야겠네”  
텍스트가 아니라 함수 호출 요청(JSON) 을 생성  

③ Execute Function Code  
실제 코드에서 get_weather("paris") 실행  
외부 API(OpenWeather 같은) 호출  

④ Results (All Prior Messages)  
함수 실행 결과가 다시 모델에게 전달  
모델은 이제 “파리의 온도 = 14도”라는 사실을 알게 됨  

⑤ Final Response  
모델이 사용자에게 자연어로 최종 답변 생성  
“It’s currently 14°C in Paris.”  

- LLM이 API를 직접 실행하는 게 아니라  
“어떤 함수를 호출할지 결정”만 하고  
실행은 개발자 코드,  
결과를 다시 받아 문장 생성  
👉 LLM + 외부 시스템 연동 구조  

In [1]:
from openai import OpenAI                 # OpenAI API를 사용하기 위한 클라이언트 클래스
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('openai_key')
OPENWEATHER_API_KEY = os.getenv('openweather_key')
OPENWEATHER_API_KEY

'c9c41a782930cfc8c78bdeb5584b16b0'

In [20]:
# OpenWeather API 호출로 부산 날씨 데이터 조회
import requests

city_name = "Busan"
units = 'metric'        # 단위 (metrics = 섭씨)
url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units={units}"

response = requests.get(url)

data = response.json()
data

{'coord': {'lon': 129.0403, 'lat': 35.1028},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 1.99,
  'feels_like': -3,
  'temp_min': 1.99,
  'temp_max': 1.99,
  'pressure': 1025,
  'humidity': 32,
  'sea_level': 1025,
  'grnd_level': 1020},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 330},
 'clouds': {'all': 0},
 'dt': 1770020336,
 'sys': {'type': 1,
  'id': 8086,
  'country': 'KR',
  'sunrise': 1769984552,
  'sunset': 1770022340},
 'timezone': 32400,
 'id': 1838524,
 'name': 'Busan',
 'cod': 200}

In [22]:
weather_info={}
if response.status_code==200:
    weather_description = data['weather'][0]['description']
    temp = data['main']['temp']
    temp_feels_like = data['main']['feels_like']
    humidity = data['main']['humidity']
    
    weather_info = {
        'city' : city_name,
        'description' : weather_description,
        'temperature' : temp,
        'temperature_feels_like' : temp_feels_like,         # 체감온도
        'humidity' : humidity                               # 습도
    }
    
else:
    weather_info = {
        'city' : city_name,
        'description' : 'Not Found',
        'temperature' : 'Not Found',
        'temperature_feels_like' : 'Not Found',
        'humidity' : 'Not Found'                  
    }
    
weather_info

{'city': 'Busan',
 'description': 'clear sky',
 'temperature': 1.99,
 'temperature_feels_like': -3,
 'humidity': 32}

In [23]:
# 실패시 실패 원이 ㄴ확인용 디버깅
print("status_code :", response.status_code)
print("response json :", data)
print("message :", data.get("message","no message"))

status_code : 200
response json : {'coord': {'lon': 129.0403, 'lat': 35.1028}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 1.99, 'feels_like': -3, 'temp_min': 1.99, 'temp_max': 1.99, 'pressure': 1025, 'humidity': 32, 'sea_level': 1025, 'grnd_level': 1020}, 'visibility': 10000, 'wind': {'speed': 6.17, 'deg': 330}, 'clouds': {'all': 0}, 'dt': 1770020336, 'sys': {'type': 1, 'id': 8086, 'country': 'KR', 'sunrise': 1769984552, 'sunset': 1770022340}, 'timezone': 32400, 'id': 1838524, 'name': 'Busan', 'cod': 200}
message : no message


In [24]:
# 날씨 API 호출 함수 : doc_string을 활용해 함수 설명
import requests
import json

def get_current_weather(city_name='Seoul', units='metric'):
    """
    OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수

    Args:
        - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**
            - 변환예시:
                - 서울 -> Seoul
                - 충남, 충청남도 -> Chungcheongnam-do
                - 부산 -> Busan
        - units: str 온도단위를 설정하는 문자열
            - metric(기본값: 섭씨, 미터)
            - imperial(화씨, 야드)
    Return:
        - str: json 형식으로 변환된 현재 날씨 정보
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units={units}"
    response = requests.get(url)

    weather_info={}
    if response.status_code==200:
        weather_description = data['weather'][0]['description']
        temp = data['main']['temp']
        temp_feels_like = data['main']['feels_like']
        humidity = data['main']['humidity']
        
        weather_info = {
            'city' : city_name,
            'description' : weather_description,
            'temperature' : temp,
            'temperature_feels_like' : temp_feels_like,         # 체감온도
            'humidity' : humidity                               # 습도
        }
        
    else:
        weather_info = {
            'city' : city_name,
            'description' : 'Not Found',
            'temperature' : 'Not Found',
            'temperature_feels_like' : 'Not Found',
            'humidity' : 'Not Found'                  
        }
        
    return json.dump(weather_info)  # dict를 JSON 문자열로 변환해 반환

In [25]:
# llm이 사용할 함수(tool) 모음
tools_to_execute = {
    'get_current_weather' : get_current_weather      # 키 : LLM이 호출할 tool 이름, 값 : 실제 실행될 함수
}

## llm 준비

In [ ]:
# get_current_weather 함수의 docstring(설명) 출력
print(tools_to_execute['get_current_weather'].__doc__)


    OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수

    Args:
        - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**
            - 변환예시:
                - 서울 -> Seoul
                - 충남, 충청남도 -> Chungcheongnam-do
                - 부산 -> Busan
        - units: str 온도단위를 설정하는 문자열
            - metric(기본값: 섭씨, 미터)
            - imperial(화씨, 야드)
    Return:
        - str: json 형식으로 변환된 현재 날씨 정보
    


In [ ]:
# Function Calling으로 날씨 API를 호출해 답변하는 대화 루프
from pprint import pprint
from openai import OpenAI

client = OpenAI()

# 사용자 질문을 받아 tool 호출 여부를 판단하고 최종 답변까지 생성하는 함수
def run_conversation(user_prompt, model='gpt-4.1-mini'):
    messages = [
        {"role" : "system", 'content' : "당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 후 대답하세요."} ,
        {"role" : "user", 'content' : user_prompt}  # 유저 프롬프트
    ],
    tools = [
        {
            'type' : 'function',
            'function' : {'name' : 'get_current_weather', 
            'description' : tools_to_execute['get_current_weather'].__doc__,
            'parameters' : {
                'type' : 'object',
                'properties' : {
                    'city_name' : {
                        'type' : 'string',
                        'description' :   """도시이름(필수값). 반드시 영어로 작성하세요.    
                                    변환예시:
                                        서울 -> Seoul
                                        충남, 충청남도 -> Chungcheongnam-do
                                        경남, 경상남도 -> Gyeongsangnam-do
                                        전남, 전라남도 -> Jeollanam-do
                                        부산 -> Busan"""                      
},
    'units' : {
            'type' : 'string',
            'description' : """온도단위를 설정하는 문자열
    
metric(기본값: 섭씨, 미터)
imperial(화씨, 야드)""",
    'enum' : ['metric', 'imperial']
        }

},
'required' : ['city_name']
                    }    
        }
        }    
    ]
